- Daten einlesen
- Texte bereinigen
- Modell(e) trainieren
- Accuracy, Precision & Recall berechnen


### Schritt 1A: Einlesen der Rohdaten
In diesem Schritt wird geprüft, ob alle Urteilstexte aus dem bereitgestellten OpenJur-Datensatz korrekt geladen werden können. Ziel ist es, die Datengrundlage zu verifizieren, bevor weitere Verarbeitungsschritte erfolgen.

In [42]:
import os
import pandas as pd
import re

DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\there\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


Die Ausgabe bestätigt, dass insgesamt 2375 Urteilstexte erfolgreich eingelesen wurden. Die Dateinamen entsprechen den von OpenJur vergebenen Fall-IDs, wodurch die Konsistenz und Vollständigkeit der Datengrundlage sichergestellt ist.

### Schritt 1B: Strukturierung der Urteilstexte
Im nächsten Schritt werden die eingelesenen Urteilstexte in eine tabellarische Datenstruktur überführt. Ziel ist es, jede gerichtliche Entscheidung als einzelne Beobachtung abzubilden und damit eine Grundlage für die spätere Merkmalsextraktion und Modellierung zu schaffen.

In [43]:
def read_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    text = read_txt(os.path.join(DATA_DIR, fn))
    rows.append({
        "case_id": case_id,
        "text": text
    })

df = pd.DataFrame(rows)
df["text_len"] = df["text"].str.len()

# Für Gerichtstyp reicht Kopfbereich (Gerichtsangabe steht dort meist)
df["head"] = df["text"].str.slice(0, 8000)

# Tenor extrahieren (Aufgabe verlangt Zielvariable aus Tenor)
def extract_tenor(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Start bei "Tenor"
    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start:
        return ""
    start = m_start.end()

    # Ende vor Tatbestand / Gründe
    m_end = re.search(
        r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b",
        text[start:], 
        flags=re.IGNORECASE
    )
    end = start + m_end.start() if m_end else min(len(text), start + 8000)

    return text[start:end].strip()

df["tenor"] = df["text"].apply(extract_tenor)

print("DataFrame shape:", df.shape)
df.head(3)



DataFrame shape: (2375, 5)


,case_id,text,text_len,head,tenor
0,2090187,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,8774,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,I. Die Klage wird abgewiesen.II. Der Kläger ha...
1,2112111,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,47492,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,"1. Die Beklagte wird verurteilt, an die klagen..."
2,2112115,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,35964,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,"Der Beklagte zu 1) wird verurteilt, an die Klä..."


Die resultierende Datenstruktur umfasst 2375 gerichtliche Entscheidungen. Jede Zeile entspricht einem Urteil, das eindeutig über eine Fall-ID referenziert ist. Zusätzlich wurde die Textlänge der Urteile berechnet, um eine Plausibilitätsprüfung der Datengrundlage zu ermöglichen.

### Schritt 1C: Einschränkung auf Entscheidungen der Landgerichte

Gemäß der Aufgabenstellung wird der Datensatz auf Urteile deutscher Landgerichte beschränkt. Hintergrund ist, dass unterschiedliche
Gerichtsebenen teils abweichende rechtliche Maßstäbe anwenden, was zu inkonsistenten Lernsituationen für das Modell führen kann. 
Zunächst wird heuristisch anhand typischer Gerichtsbezeichnungen im Kopfbereich der Entscheidung geprüft, ob es sich um ein Urteil eines deutschen Landgerichts handelt. Anschließend wird der thematische Bezug zum Dieselskandal anhand zentraler Schlüsselbegriffe identifiziert. Durch diese getrennte Markierung wird Transparenz über die Zusammensetzung des Datensatzes geschaffen und eine spätere Anpassung der Filterlogik ermöglicht, ohne frühzeitig Daten zu verwerfen.

In [44]:
lg_pattern = r"\bLandgericht\b|\bLG\s+[A-ZÄÖÜa-zäöü]+"

df["is_landgericht"] = df["head"].str.contains(lg_pattern, case=False, regex=True)

df["is_landgericht"].value_counts()


is_landgericht
True     2088
False     287
Name: count, dtype: int64

In [45]:
# Diesel-Fälle gemäß Aufgabenstellung
diesel_keywords = ["EA188", "EA189", "Abgasskandal", "Dieselskandal"]
diesel_pattern = "|".join(map(re.escape, diesel_keywords))

df["is_diesel_case"] = df["text"].str.contains(
    diesel_pattern, case=False, regex=True
)

df["is_diesel_case"].value_counts()



is_diesel_case
True     2336
False      39
Name: count, dtype: int64

### Schritt 1D: Finaler Analyse-Datensatz 
Der finale Analyse-Datensatz umfasst ausschließlich dieselbezogene Landgerichtsurteile und dient als Grundlage für die weitere Modellierung.

In [46]:
df_analysis = df[df["is_diesel_case"] & df["is_landgericht"]].copy()
df_analysis.shape


(2056, 7)

Zur methodischen Absicherung wurde überprüft, ob alle Dokumente entsprechende Schlüsselbegriffe enthalten. Die Analyse bestätigt, dass nahezu alle Urteile einen expliziten Bezug zum Dieselskandal aufweisen.

### Schritt 2: Definition der Zielvariable

Ziel der Analyse ist die Vorhersage, ob in einem Urteil ein Schadensersatz zugesprochen wurde oder nicht. 
Die Zielvariable wird binär modelliert (1 = Schadensersatz zugesprochen, 0 = kein Schadensersatz). 
Da strukturierte Labels nicht vorliegen, erfolgt die Ableitung regelbasiert anhand typischer Formulierungen im Tenor der Entscheidung.



In [47]:
def infer_target(tenor: str):
    if not isinstance(tenor, str) or tenor.strip() == "":
        return None

    t = tenor.lower()

    # Verurteilung zur Zahlung → Schadensersatz
    positive_patterns = [
        r"wird verurteilt.*\bzu zahlen\b",
        r"wird verurteilt.*\ban den kläger\b.*\bzu zahlen\b",
        r"wird verurteilt.*\ban die klägerin\b.*\bzu zahlen\b",
        r"\bzu zahlen\b.*(\bEUR\b|€)",
        r"(\bEUR\b|€).*?\bzu zahlen\b",
        r"schadensersatz"
    ]
    if any(re.search(p, t, flags=re.DOTALL) for p in positive_patterns):
        return 1

    # Klage abgewiesen → kein Schadensersatz
    negative_patterns = [
        "die klage wird abgewiesen",
        "klage wird abgewiesen",
        "die berufung wird zurückgewiesen",
        "berufung wird zurückgewiesen",
        "wird zurückgewiesen",
        "wird verworfen",
        "als unzulässig verworfen",
        "wird als unzulässig verworfen",
    ]
    if any(p in t for p in negative_patterns):
        return 0

    return None


df_analysis["target"] = df_analysis["tenor"].apply(infer_target)
df_analysis["target"].value_counts(dropna=False)


target
0.0    1026
1.0     715
NaN     315
Name: count, dtype: int64

In [48]:
df_analysis[df_analysis["target"].notna()][["case_id","target","tenor"]].sample(20, random_state=42)



,case_id,target,tenor
704,2274949,0.0,1. Die Berufung der Klagepartei gegen das Urte...
2074,2453299,0.0,1. Die Klage wird abgewiesen.2. Die Kosten des...
1319,2343116,0.0,1. Die Berufung des Klägers gegen das Urteil d...
1396,2350176,0.0,Die Berufung der Klägerin gegen das am 30. Okt...
984,2306384,1.0,Auf die Berufung der Beklagten wird das am 31....
398,2205120,0.0,Die Klage wird abgewiesen.Die Klägerin trägt d...
636,2270509,0.0,Die Klage wird abgewiesen.Die Kosten des Recht...
1825,2394941,0.0,Die Berufung des Klägers gegen das am 16. Apri...
1586,2378432,0.0,1. Die Klage wird abgewiesen.2. Der Kläger hat...
858,2297195,0.0,1. Auf die Berufung der Beklagten wird das End...


Die regelbasierte Ableitung der Zielvariable wurde anhand einer zufälligen Stichprobe überprüft, wobei ausschließlich der jeweilige Tenor betrachtet wurde.

**Umgang mit unklaren Fällen**

Nicht alle Urteile enthalten eine eindeutig identifizierbare Tenorformulierung, aus der zweifelsfrei hervorgeht, ob ein Schadensersatz zugesprochen wurde oder nicht (z. B. bei Vergleichen oder rein prozessualen Entscheidungen).  
Diese Fälle werden in der Zielvariable als *unklar* (`None`) gekennzeichnet und für die weitere Modellierung ausgeschlossen.  
Durch diese Einschränkung wird sichergestellt, dass das Modell ausschließlich auf eindeutig gelabelten Entscheidungen trainiert wird und Verzerrungen durch unsichere Zuordnungen vermieden werden.

##Schritt 3: KI-gestützte Merkmalsextraktion mit Gemini 

In [49]:
def build_prompt(head: str, tenor: str) -> str:
    head = (head or "").strip()
    tenor = (tenor or "").strip()

    return f"""
Du bist ein juristischer Analyseassistent.

AUFGABE:
Analysiere einen deutschen Gerichtsentscheid anhand von Kopf und Tenor.

1) Gerichtstyp:
- Bestimme, ob es sich um ein Landgericht (LG) oder ein Oberlandesgericht (OLG) handelt.
- Gib ausschließlich "LG" oder "OLG" zurück (keine Stadt, keine Zusatzinfos).

2) Entscheidung:
- Prüfe im TENOR, ob ein Schadensersatzbetrag zugesprochen wird.
  → Gib den Betrag exakt als String zurück, z.B. "23.542,23 EUR".
- Falls die Klage abgewiesen wurde:
  → Gib exakt "Klage abgewiesen" zurück.
- Falls weder Betrag noch Abweisung eindeutig genannt sind:
  → Gib "Sonstige: <kurzer Grund>" zurück.

WICHTIGE REGELN:
- Extrahiere NICHT den Streitwert.
- Interpretiere NUR den Tenor.
- Erfinde keine Beträge.
- Antworte AUSSCHLIESSLICH im JSON-Format.

FORMAT (GENAU EIN ELEMENT):
[
  {{
    "Gerichtstyp": "LG oder OLG",
    "Urteil": "<Betrag in EUR | Klage abgewiesen | Sonstige: ...>"
  }}
]

KOPF:
\"\"\"{head[:800]}\"\"\"

TENOR:
\"\"\"{tenor[:6000]}\"\"\"
""".strip()

In [54]:
# ==========================================
# JSONL Generator für Gemini Batch Upload
# Voraussetzung:
# - df_analysis existiert (case_id, head, tenor)
# - build_prompt(head, tenor) existiert
# - MAX_CHARS existiert
# ==========================================

import json
from pathlib import Path
import pandas as pd

# ---- Steuerung ----
RUN_MODE = "test"      # "test" oder "full"
TEST_LIMIT = 1000
HARD_CAP = 3000        # Safety: nie mehr als das (auch in full)

# ---- Output ----
OUT_DIR = Path("../jupyter_notebook")
OUT_DIR.mkdir(parents=True, exist_ok=True)

JSONL_PATH = OUT_DIR / ("batch_requests_test.jsonl" if RUN_MODE == "test" else "batch_requests_full.jsonl")

# ---- Worklist bauen ----
df_work = df_analysis.copy()
df_work["case_id"] = df_work["case_id"].astype(str)

# Optional: Resume-Filter, falls du done_ids hast:
# done_ids = set(...)  # aus vorhandenen Ergebnissen
# df_work = df_work[~df_work["case_id"].isin(done_ids)].copy()

if RUN_MODE == "test":
    df_work = df_work.head(TEST_LIMIT).copy()
elif RUN_MODE == "full":
    if len(df_work) > HARD_CAP:
        df_work = df_work.head(HARD_CAP).copy()
else:
    raise ValueError("RUN_MODE muss 'test' oder 'full' sein")

print(f"✅ JSONL Mode={RUN_MODE} | Fälle={len(df_work)} | Datei={JSONL_PATH.resolve()}")

# ---- JSONL schreiben ----
with open(JSONL_PATH, "w", encoding="utf-8") as f:
    for r in df_work.itertuples(index=False):
        case_id = getattr(r, "case_id")
        head = getattr(r, "head", "") if hasattr(r, "head") else ""
        tenor = getattr(r, "tenor", "") if hasattr(r, "tenor") else ""

        prompt = build_prompt(head, tenor)[:MAX_CHARS]

        # Generisches Request-Format (leicht anpassbar an deine Batch-API)
        obj = {
            "key": str(case_id),
  "request": {
    "contents": [
      {"role": "user", "parts": [{"text": prompt}]}
    ]
    # optional: "generation_config": {...}
  }         # Prompt ist drin (Batch kennt dein Notebook nicht!)
        }

        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print("🎉 Fertig geschrieben:", JSONL_PATH.resolve())

# ---- Mini-Check: erste Zeile anzeigen ----
with open(JSONL_PATH, "r", encoding="utf-8") as f:
    first_line = f.readline().strip()
print("Erste Zeile (gekürzt):", first_line[:300])

✅ JSONL Mode=test | Fälle=1000 | Datei=C:\Users\there\ds_law\backend\jupyter_notebook\batch_requests_test.jsonl
🎉 Fertig geschrieben: C:\Users\there\ds_law\backend\jupyter_notebook\batch_requests_test.jsonl
Erste Zeile (gekürzt): {"key": "2090187", "request": {"contents": [{"role": "user", "parts": [{"text": "Du bist ein juristischer Analyseassistent.\n\nAUFGABE:\nAnalysiere einen deutschen Gerichtsentscheid anhand von Kopf und Tenor.\n\n1) Gerichtstyp:\n- Bestimme, ob es sich um ein Landgericht (LG) oder ein Oberlandesgeric


In [51]:
import google.generativeai as genai
import json
import time
import pandas as pd

genai.configure(api_key="GEMINI_API_KEY")
model = genai.GenerativeModel('gemini-1.5-flash')

def extrahiere_infos(row):
    text_input = f"Kopf: {row['head']}\nTenor: {row['tenor']}"
    prompt = """
    Analysiere das Urteil und gib EXKLUSIV ein JSON-Objekt zurück mit:
    - 'Gerichtstyp': (LG oder OLG)
    - 'Urteil': (Betrag als Zahl oder 'Klage abgewiesen')
    - 'Motor': (z.B. EA189, EA288)
    - 'Anspruch': (Ja oder Nein)
    """
    try:
        response = model.generate_content(text_input + prompt)
        res_text = response.text.replace('```json', '').replace('```', '').strip()
        return json.loads(res_text)
    except:
        return None

# Testlauf mit 5 Urteilen
df_test = df[df["is_landgericht"] == True].head(5).copy()
results = []

for index, row in df_test.iterrows():
    print(f"KI analysiert Urteil {row['case_id']}...")
    daten = extrahiere_infos(row)
    if daten:
        daten['case_id'] = row['case_id']
        results.append(daten)
    time.sleep(1)

# Tabelle anzeigen
df_final = pd.DataFrame(results)
df_final

KI analysiert Urteil 2090187...
KI analysiert Urteil 2112111...
KI analysiert Urteil 2112115...
KI analysiert Urteil 2112117...
KI analysiert Urteil 2112118...


""
